In [63]:
#importamos la base de datos
import pandas as pd #pandas
df=pd.read_csv('airbnb.csv',encoding='latin-1')
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [64]:
#importamos todas las librerías necesarias: 
import sklearn #scikit-learn
import matplotlib.pyplot as plt #gráfica Matplotlib
import numpy as np #NumPy

Primero debemos ver qué hacemos con los datos NaN para tratarlos

Analizamos cuántos datos NaN hay en la columna reviews_per_month


In [65]:
reviews_per_month_nan = df['reviews_per_month'].isna().sum()
reviews_per_month_nan

10052

Ahora bien, vemos que hay 10052 datos NaN en la columna. Observemos cuántos datos hay en total para barajar las distintas opciones de qué hacer con estos datos NaN. Si es poca cantidad comparado con la cantidad de observaciones en total, podremos eliminar directamente esas observaciones, sin embargo, si suponen gran cantidad, podríamos buscar otras opciones, como rellenar con la media para así no afectar al equilibrio del resto de datos. 
También es verdad que como hablamos de reseñas por mes, sería factible convertir esos NaN en 0, para que así a la hora de categorizar la columna, lo detecte como Bajo. 

In [66]:
num_observations = df.shape[0]
num_observations

48895

Supone un 20% de los datos totales. No resulta útil borrarlos, si no que deberíamos rellenar con 0's. 

In [67]:
df['reviews_per_month'].fillna(0, inplace=True)

¿Por qué rellenamos con 0's? 
Encontramos un NaN en esa columna, cuando en la columna de number_of_reviews aparece un 0, o un NaN directamente, por lo que no sería "justo" por así decirlo rellenar con la media, ya que ahí estarías metiendo posibles "evaluaciones" que no existen. 

In [68]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,0.00,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


Comprobamos por tanto que en esa columna ya no nos quede ningún NaN

In [69]:
reviews_per_month_nan = df['reviews_per_month'].isna().sum()
reviews_per_month_nan

0

Ya nos hemos asegurado que no nos quedad ningún dato NaN, así no encontraremos ningún error a la hora de categorizar la columna. 

Si no también otra manera de hacerlo es: 
calculando a mano 3 rangos, ver cuál es el mayor, el menos y dividir en 3 rangos. 


-Crea una lista de intervalos de tres categorías

bins = [0, 3, 6, np.inf]

es un ejemplo de los rangos, los debemos calcular bien. 

-Asigna una categoría a cada valor en la columna 'reviews_per_month'

df['reviews_per_month'] = pd.cut(df['reviews_per_month'], bins, labels=['Baja', 'Media', 'Alta'])

In [70]:
df['reviews_per_month'].max()

58.5

In [71]:
index_max = df['reviews_per_month'].idxmax()
index_max

42075

In [72]:
print(df['reviews_per_month'].head(index_max))

0        0.21
1        0.38
2        0.00
3        4.64
4        0.10
         ... 
42070    1.80
42071    1.69
42072    0.68
42073    7.08
42074    0.00
Name: reviews_per_month, Length: 42075, dtype: float64


In [73]:
# Define el número de intervalos
num_intervalos = 3

# Crea una lista de intervalos
intervalos = np.linspace(df['reviews_per_month'].min(), df['reviews_per_month'].max(), num_intervalos+1)

# Creamos las etiquetas 
etiquetas = ['Baja', 'Media', 'Alta']

# Asigna una categoría a cada intervalo
df['reviews_per_month'] = pd.cut(df['reviews_per_month'], intervalos, labels=etiquetas)

In [75]:
print(df['reviews_per_month'].head(index_max+1))

0        Baja
1        Baja
2         NaN
3        Baja
4        Baja
         ... 
42071    Baja
42072    Baja
42073    Baja
42074     NaN
42075    Alta
Name: reviews_per_month, Length: 42076, dtype: category
Categories (3, object): ['Baja' < 'Media' < 'Alta']


Parece ser que sí que detecta los altos, pensé que ponía todos como bajos

No lo está haciendo bien, habíamos quitado los NaN supuestamente, y ahora vuelven a aparecer. 

OTRA MANERA

ALGO DE ESTO VA EN LO DE CATEGORIZAR LA COLUMNA EN 3 CATEGORÍAS


from sklearn.preprocessing import OrdinalEncoder

-Creamos el codificador indicandole el orden de la variables

encoder = OrdinalEncoder(categories=[["Baja", "Media", 
                                      "Alta"]])

-Ajustamos el codificador con la variable education y la transformamos

encoder.fit(datos[["reviews_per_month"]])

datos["reviews_per_month"] = encoder.transform(df[["reviews_per_month"]])

In [ ]:
#vamos a categorizar la columna reviews_per_month en 3 categorías: “Baja”, “Media” y ”Alta”
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[["Baja", "Media", "Alta"]])
encoder.fit(datos[["reviews_per_month"]]) 
df["reviews_per_month"] = encoder.transform(df[["reviews_per_month"]])


ValueError: ignored